# Imports

In [ ]:
import random
import copy
import math

# Declaring Functions

## Parse graph from .col file (Dataset)

In [2]:
def parse_col_file(filepath):
    adj = {}
    num_vertices = 0

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('c'):
                continue

            if line.startswith('p'):
                _, _, n, _ = line.split()
                num_vertices = int(n)
                adj = {i: set() for i in range(1, num_vertices + 1)}

            elif line.startswith('e'):
                _, u, v = line.split()
                u, v = int(u), int(v)
                adj[u].add(v)
                adj[v].add(u)

    return adj, num_vertices


## Fitness and Conflict Evaluation

In [3]:
def count_conflicts(adj, coloring):
    conflicts = 0
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts += 1
    return conflicts

In [4]:
def num_colors_used(coloring):
    return len(set(coloring.values()))

In [5]:
def fitness(adj, coloring, alpha=1000, beta=1):
    return alpha * count_conflicts(adj, coloring) + beta * num_colors_used(coloring)

## Population Init

In [6]:
def random_coloring(num_vertices, max_colors):
    return {i: random.randint(1, max_colors) for i in range(1, num_vertices + 1)}

In [7]:
def initialize_population(pop_size, num_vertices, max_colors):
    return [random_coloring(num_vertices, max_colors) for _ in range(pop_size)]

## Tournament Selection

In [8]:
def tournament_selection(population, adj, k=3):
    candidates = random.sample(population, k)
    return min(candidates, key=lambda c: fitness(adj, c))

## Crossover (Uniform)

In [9]:
def uniform_crossover(parent1, parent2):
    child = {}
    for v in parent1:
        child[v] = parent1[v] if random.random() < 0.5 else parent2[v]
    return child


## Mutation and conflict fix

In [10]:
def mutate(coloring, adj, max_colors, mutation_rate=0.02):
    for v in coloring:
        if random.random() < mutation_rate:
            coloring[v] = random.randint(1, max_colors)

    # Extra: fix conflicts
    for u in adj:
        for v in adj[u]:
            if coloring[u] == coloring[v]:
                coloring[v] = random.randint(1, max_colors)

# Genetic Algorithm

## Definition of the Genetic Function

In [11]:
def genetic_algorithm(
    adj,
    num_vertices,
    max_colors,
    pop_size=100,
    generations=1000,
    elitism_rate=0.05,
    mutation_rate=0.02
):
    population = initialize_population(pop_size, num_vertices, max_colors) # Initial population
    elite_size = int(pop_size * elitism_rate) # Number of elites

    best_solution = None
    best_fitness = float('inf')
    found_feasible = False
    for gen in range(generations):      # Main GA loop
        population.sort(key=lambda c: fitness(adj, c))  # Sort by fitness
        elites = population[:elite_size] # Select elites to carry over unchanged to next generation

        if fitness(adj, population[0]) < best_fitness: # Update best solution found
            best_solution = copy.deepcopy(population[0])
            best_fitness = fitness(adj, population[0])

        if count_conflicts(adj, population[0]) == 0 and not found_feasible: # Found first feasible solution
            print(f"Feasible solution found at generation {gen}")
            
            found_feasible = True

        if gen % 50 == 0: # Print status every 50 generations
            print(
                f"Gen {gen} | Best fitness: {best_fitness} | "
                f"Conflicts: {count_conflicts(adj, best_solution)} | "
                f"Colors: {num_colors_used(best_solution)}"
            )
        
        
        new_population = elites.copy() # Start new population with elites

        while len(new_population) < pop_size:         # Generate rest of population
            p1 = tournament_selection(population, adj)
            p2 = tournament_selection(population, adj)

            child = uniform_crossover(p1, p2)
            mutate(child, adj, max_colors, mutation_rate)

            new_population.append(child)

        population = new_population


    return best_solution


## Usage of the Genetic Algorithm

In [12]:
if __name__ == "__main__":
    adj, n = parse_col_file("queen5_5.col")

    MAX_COLORS = 12

    solution = genetic_algorithm(
        adj=adj,
        num_vertices=n,
        max_colors=MAX_COLORS,
        pop_size=200,
    )

    print("Final solution:")
    print("Conflicts:", count_conflicts(adj, solution))
    print("Colors used:", num_colors_used(solution))


Gen 0 | Best fitness: 6010 | Conflicts: 6 | Colors: 10
Feasible solution found at generation 1
Gen 50 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 100 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 150 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 200 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 250 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 300 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 350 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 400 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 450 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 500 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 550 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 600 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 650 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 700 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 750 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 800 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 850 | Best fitness: 7 | Conflicts: 0 | Colors: 7
Gen 9

# Tabu Search

## Implementation of an optimized conflict storage
Using a data structure reduces the amount of computations since the tabu search requires not only the amount of conflicts but where have they appeared in order to generate meaningful neighborhood moves, because only recoloring conflicted vertices can reduce conflicts.
The total conflict count alone provides no structural information about where the problem is, making targeted local search impossible.

In [13]:
def conflicting_vertices(adj, coloring):
    conflicts = set()
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts.add(u)
                conflicts.add(v)
    return list(conflicts)


## Implementation of the algorithm

In [ ]:
def tabu_search(
    adj,
    num_vertices,
    max_colors,
    max_iterations=20000,
    tabu_tenure=7,
    aspiration=True,
):
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)

    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors

    tabu_list = {}
    iteration = 0

    while iteration < max_iterations:

        conflicted = conflicting_vertices(adj, current)
        best_move = None
        best_score = (float("inf"), float("inf"))

        for v in conflicted:
            original_color = current[v]

            for c in range(1, max_colors + 1):
                if c != original_color:

                    current[v] = c
                    new_conflicts = count_conflicts(adj, current)
                    new_colors = num_colors_used(current)

                    score = (new_conflicts, new_colors)
                    delta_better = score < best_score

                    move = (v, c)
                    tabu = move in tabu_list and tabu_list[move] > iteration

                    if tabu and aspiration:
                        if (new_conflicts < best_conflicts or
                           (new_conflicts == best_conflicts and new_colors < best_colors)):
                            tabu = False

                    if not tabu and delta_better:
                        best_score = score
                        best_move = (v, c, new_conflicts, new_colors)

                    current[v] = original_color

        if best_move is not None:
            v, c, new_conflicts, new_colors = best_move
            old_color = current[v]

            current[v] = c
            current_conflicts = new_conflicts
            current_colors = new_colors

            tabu_list[(v, old_color)] = iteration + tabu_tenure

            # Update global best using lexicographic objective
            if (current_conflicts < best_conflicts or
               (current_conflicts == best_conflicts and current_colors < best_colors)):
                best = copy.deepcopy(current)
                best_conflicts = current_conflicts
                best_colors = current_colors

        if iteration % 500 == 0:
            print(
                f"Iter {iteration} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )

        iteration += 1

    return best


## Usage of Tabu Search

In [25]:
if __name__ == "__main__":
    adj, n = parse_col_file("le450_15b.col")

    MAX_COLORS = 15

    solution = tabu_search(
        adj=adj,
        num_vertices=n,
        max_colors=MAX_COLORS,
        max_iterations=2000,
        tabu_tenure=7,
    )

    print("Final solution:")
    print("Conflicts:", count_conflicts(adj, solution))
    print("Colors used:", num_colors_used(solution))


Iter 0 | Conflicts: 463 | Colors: 15 | Best colors: 15
Iter 500 | Conflicts: 68 | Colors: 15 | Best colors: 15
Iter 1000 | Conflicts: 68 | Colors: 15 | Best colors: 15
Iter 1500 | Conflicts: 68 | Colors: 15 | Best colors: 15
Final solution:
Conflicts: 68
Colors used: 15


# Simmulated Annealing

## Algorithm implementation

In [ ]:
def simulated_annealing(
    adj,
    num_vertices,
    max_colors,
    max_iterations=50000,
    initial_temp=10.0,
    cooling_rate=0.9995,
    conflict_weight=1000,
):
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)

    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors

    temperature = initial_temp
    iteration = 0

    while iteration < max_iterations and temperature > 1e-6:

        # Pick a vertex biased toward conflicts
        conflicted = conflicting_vertices(adj, current)
        if len(conflicted) > 0:
            v = random.choice(conflicted)
        else:
            v = random.randrange(num_vertices)

        original_color = current[v]
        new_color = random.randint(1, max_colors)

        if new_color != original_color:

            current[v] = new_color
            new_conflicts = count_conflicts(adj, current)
            new_colors = num_colors_used(current)

            delta = (
                (new_conflicts - current_conflicts) * conflict_weight
                + (new_colors - current_colors)
            )

            accept = False
            if delta <= 0:
                accept = True
            else:
                prob = math.exp(-delta / temperature)
                accept = random.random() < prob

            if accept:
                current_conflicts = new_conflicts
                current_colors = new_colors

                if (
                    current_conflicts < best_conflicts or
                    (current_conflicts == best_conflicts and current_colors < best_colors)
                ):
                    best = copy.deepcopy(current)
                    best_conflicts = current_conflicts
                    best_colors = current_colors
            else:
                current[v] = original_color

        temperature *= cooling_rate
        iteration += 1

        if iteration % 1000 == 0:
            print(
                f"Iter {iteration} | "
                f"T={temperature:.4f} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )

    return best


## Running the algorithm


In [ ]:
if __name__ == "__main__":
    adj, n = parse_col_file("queen5_5.col")

    MAX_COLORS = 12

    solution = simulated_annealing(
        adj=adj,
        num_vertices=n,
        max_colors=MAX_COLORS,
    )

    print("Final solution:")
    print("Conflicts:", count_conflicts(adj, solution))
    print("Colors used:", num_colors_used(solution))
